In [1]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime


path = 'Kids'
images = []
personNames = []
myList = os.listdir(path)
myList = myList[::2]
print(myList)
for cu_img in myList:
    current_Img = cv2.imread(f'{path}/{cu_img}')
    images.append(current_Img)
    personNames.append(os.path.splitext(cu_img)[0])
print(personNames)


def faceEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

['Kevin.jpg', 'Hermione.jpg', 'Harry.jpg']
['Kevin', 'Hermione', 'Harry']


In [2]:
encodeListKnown = faceEncodings(images)
print('All Encodings Complete!!!')

All Encodings Complete!!!


In [5]:
cap = cv2.VideoCapture(0)
opc_count = 0
object_id_list = []
while True:
    ret, frame = cap.read()
    faces = cv2.resize(frame, (0, 0), None, fx = 0.25, fy = 0.25)
    faces = cv2.cvtColor(faces, cv2.COLOR_BGR2RGB)

    facesCurrentFrame = face_recognition.face_locations(faces)
    encodesCurrentFrame = face_recognition.face_encodings(faces, facesCurrentFrame)
    i=0
    for encodeFace, faceLoc in zip(encodesCurrentFrame, facesCurrentFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        # print(faceDis)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = personNames[matchIndex].upper()
            # print(name)
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            cv2.rectangle(frame, (x1, y2 + 20), (x2, y2), (0, 0, 255), cv2.FILLED)
            i = i+1
            cv2.putText(frame, name, (x1 + 3, y2 + 14), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255), 2)
            #attendance(name)
            
            if i not in object_id_list:
                object_id_list.append(i)
    opc_count = len(object_id_list)
    opc_text = ("MISSING KID SPOTTED:{}".format(opc_count))
    cv2.putText(frame, opc_text,(5,30), cv2.FONT_HERSHEY_DUPLEX, 1, (255,255,255), 2)

    cv2.imshow('Webcam', frame)
    if cv2.waitKey(1) == 13:
        break

cap.release()
cv2.destroyAllWindows()